In [29]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg

In [30]:
print("The number of images with facemask labelled 'yes':",len(os.listdir('facemask-dataset/yes')))
print("The number of images with facemask labelled 'no':",len(os.listdir('facemask-dataset/no')))

The number of images with facemask labelled 'yes': 690
The number of images with facemask labelled 'no': 686


In [31]:
def data_summary(main_path):
    
    yes_path = main_path+'yesreal'
    no_path = main_path+'noreal'
        
    # number of files (images) that are in the the folder named 'yes' that represent tumorous (positive) examples
    m_pos = len(listdir(yes_path))
    # number of files (images) that are in the the folder named 'no' that represent non-tumorous (negative) examples
    m_neg = len(listdir(no_path))
    # number of all examples
    m = (m_pos+m_neg)
    
    pos_prec = (m_pos* 100.0)/ m
    neg_prec = (m_neg* 100.0)/ m
    
    print(f"Number of examples: {m}")
    print(f"Percentage of positive examples: {pos_prec}%, number of pos examples: {m_pos}") 
    print(f"Percentage of negative examples: {neg_prec}%, number of neg examples: {m_neg}") 
    
augmented_data_path = 'facemask-dataset/trial1/augmented data1/'    
data_summary(augmented_data_path)

Number of examples: 2752
Percentage of positive examples: 50.145348837209305%, number of pos examples: 1380
Percentage of negative examples: 49.854651162790695%, number of neg examples: 1372


In [32]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    dataset = []
    
    for unitData in os.listdir(SOURCE):
        data = SOURCE + unitData
        if(os.path.getsize(data) > 0):
            dataset.append(unitData)
        else:
            print('Skipped ' + unitData)
            print('Invalid file i.e zero size')
    
    train_set_length = int(len(dataset) * SPLIT_SIZE)
    test_set_length = int(len(dataset) - train_set_length)
    shuffled_set = random.sample(dataset, len(dataset))
    train_set = dataset[0:train_set_length]
    test_set = dataset[-test_set_length:]
       
    for unitData in train_set:
        temp_train_set = SOURCE + unitData
        final_train_set = TRAINING + unitData
        copyfile(temp_train_set, final_train_set)
    
    for unitData in test_set:
        temp_test_set = SOURCE + unitData
        final_test_set = TESTING + unitData
        copyfile(temp_test_set, final_test_set)
        
        
YES_SOURCE_DIR = "facemask-dataset/trial1/augmented data1/yesreal/"
TRAINING_YES_DIR = "facemask-dataset/trial1/augmented data1/training/yes1/"
TESTING_YES_DIR = "facemask-dataset/trial1/augmented data1/testing/yes1/"
NO_SOURCE_DIR = "facemask-dataset/trial1/augmented data1/noreal/"
TRAINING_NO_DIR = "facemask-dataset/trial1/augmented data1/training/no1/"
TESTING_NO_DIR = "facemask-dataset/trial1/augmented data1/testing/no1/"
split_size = .8
split_data(YES_SOURCE_DIR, TRAINING_YES_DIR, TESTING_YES_DIR, split_size)
split_data(NO_SOURCE_DIR, TRAINING_NO_DIR, TESTING_NO_DIR, split_size)

In [33]:
print("The number of images with facemask in the training set labelled 'yes':", len(os.listdir('facemask-dataset/trial1/augmented data1/training/yes1')))
print("The number of images with facemask in the test set labelled 'yes':", len(os.listdir('facemask-dataset/trial1/augmented data1/testing/yes1')))
print("The number of images without facemask in the training set labelled 'no':", len(os.listdir('facemask-dataset/trial1/augmented data1/training/no1')))
print("The number of images without facemask in the test set labelled 'no':", len(os.listdir('facemask-dataset/trial1/augmented data1/testing/no1')))

The number of images with facemask in the training set labelled 'yes': 1104
The number of images with facemask in the test set labelled 'yes': 276
The number of images without facemask in the training set labelled 'no': 1097
The number of images without facemask in the test set labelled 'no': 275


In [34]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [35]:
TRAINING_DIR = "facemask-dataset/trial1/augmented data1/training"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "facemask-dataset/trial1/augmented data1/testing"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

Found 2201 images belonging to 2 classes.
Found 551 images belonging to 2 classes.


In [36]:
history = model.fit_generator(train_generator,
                              epochs=30,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/30


<ipython-input-36-fb50dbaa623d>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


221/221 [==============================] - ETA: 0s - loss: 0.5027 - acc: 0.7583INFO:tensorflow:Assets written to: model-001.model\assets


INFO:tensorflow:Assets written to: model-001.model\assets


221/221 [==============================] - 99s 442ms/step - loss: 0.5027 - acc: 0.7583 - val_loss: 0.2308 - val_acc: 0.9093
Epoch 2/30
221/221 [==============================] - ETA: 0s - loss: 0.2862 - acc: 0.8937INFO:tensorflow:Assets written to: model-002.model\assets


...

221/221 [==============================] - 109s 493ms/step - loss: 0.0488 - acc: 0.9855 - val_loss: 0.0238 - val_acc: 0.9946
Epoch 19/30
221/221 [==============================] - 107s 485ms/step - loss: 0.1030 - acc: 0.9664 - val_loss: 0.0266 - val_acc: 0.9946
Epoch 20/30
221/221 [==============================] - 107s 485ms/step - loss: 0.1004 - acc: 0.9668 - val_loss: 0.1215 - val_acc: 0.9583
Epoch 21/30
221/221 [==============================] - 108s 491ms/step - loss: 0.0845 - acc: 0.9750 - val_loss: 0.0255 - val_acc: 0.9946
Epoch 22/30
221/221 [==============================] - ETA: 0s - loss: 0.0534 - acc: 0.9827INFO:tensorflow:Assets written to: model-022.model\assets


INFO:tensorflow:Assets written to: model-022.model\assets


221/221 [==============================] - 109s 493ms/step - loss: 0.0534 - acc: 0.9827 - val_loss: 0.0225 - val_acc: 0.9927
Epoch 23/30
221/221 [==============================] - 110s 495ms/step - loss: 0.0755 - acc: 0.9786 - val_loss: 0.0478 - val_acc: 0.9819
Epoch 24/30
221/221 [==============================] - 107s 482ms/step - loss: 0.1459 - acc: 0.9491 - val_loss: 0.0611 - val_acc: 0.9782
Epoch 25/30
221/221 [==============================] - 107s 485ms/step - loss: 0.1114 - acc: 0.9596 - val_loss: 0.1105 - val_acc: 0.9437
Epoch 26/30
221/221 [==============================] - ETA: 0s - loss: 0.0777 - acc: 0.9755INFO:tensorflow:Assets written to: model-026.model\assets


INFO:tensorflow:Assets written to: model-026.model\assets


221/221 [==============================] - 110s 495ms/step - loss: 0.0777 - acc: 0.9755 - val_loss: 0.0210 - val_acc: 0.9946
Epoch 27/30
221/221 [==============================] - 107s 485ms/step - loss: 0.0565 - acc: 0.9832 - val_loss: 0.0460 - val_acc: 0.9837
Epoch 28/30
221/221 [==============================] - 110s 496ms/step - loss: 0.0512 - acc: 0.9832 - val_loss: 0.0765 - val_acc: 0.9746
Epoch 29/30
221/221 [==============================] - 107s 482ms/step - loss: 0.0729 - acc: 0.9732 - val_loss: 0.1135 - val_acc: 0.9655
Epoch 30/30
221/221 [==============================] - 107s 486ms/step - loss: 0.0805 - acc: 0.9777 - val_loss: 0.0445 - val_acc: 0.9927


In [37]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [45]:
#model = tf.keras.models.load_model('model-017.model')
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(5)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()

1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 28ms/step
